# Table of Contents
#### 1. Importing Libraries
#### 2. Importing Data
#### 3. Question 2: consider any security implications that might exist
#### 4. Question 3: Create regions
#### 5. Question 4: Create an exclusion flag for low-activity customers and exclude them from the data
#### 6. Question 5: Create a profiling variable based on age, income, certain goods in the 'department_id' column, and number of dependents
#### 6.1 Creating age flag
#### 6.2 Creating an income flag

## 1. Importing Libraries

In [2]:
import pandas as pd
import numpy as np
import os

## 2. Importing Data

In [3]:
path = r'C:\Users\bees\Instacart Basket Analysis'

In [4]:
ords_prods_cust = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_customers_combined.pkl'))

## 3. Question 2:
#### Consider any security implications that might exist for this new data

In [5]:
ords_prods_cust.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,order_frequency_flag,first_name,last_name,gender,state,age,date_joined,number_of_dependants,marital_status,income
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423


In [6]:
# dropping the first_name and last_name columns
ords_prods_cust = ords_prods_cust.drop(columns = ['first_name', 'last_name'])

In [7]:
ords_prods_cust.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,spending_flag,median_order,order_frequency_flag,gender,state,age,date_joined,number_of_dependants,marital_status,income
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423


In [8]:
pd.set_option('display.max_columns', None)

## 4. Question 3:
#### Create a regional segmentation of the data

In [9]:
# creating regions
Northeast = ['Maine', 'New Hampshire', 'Vermont', 'Massachusetts', 'Rhode Island', 'Connecticut', 'New York', 'Pennsylvania', 'New Jersey']
Midwest = ['Wisconsin', 'Michigan', 'Illinois', 'Indiana', 'Ohio', 'North Dakota', 'South Dakota', 'Nebraska', 'Kansas', 'Minnesota', 'Iowa', 'Missouri']
South = ['Delaware', 'Maryland', 'District of Columbia', 'Virginia', 'West Virginia', 'North Carolina', 'South Carolina', 'Georgia', 'Florida', 'Kentucky', 'Tennessee', 'Mississippi', 'Alabama', 'Oklahoma', 'Texas', 'Arkansas', 'Louisiana']
West = ['Idaho', 'Montana', 'Wyoming', 'Nevada', 'Utah', 'Colorado', 'Arizona', 'New Mexico', 'Alaska', 'Washington', 'Oregon', 'California', 'Hawaii']

In [10]:
# creating column with the regions
ords_prods_cust.loc[ords_prods_cust['state'].isin(Northeast), 'region'] = 'Northeast'
ords_prods_cust.loc[ords_prods_cust['state'].isin(Midwest), 'region'] = 'Midwest'
ords_prods_cust.loc[ords_prods_cust['state'].isin(South), 'region'] = 'South'
ords_prods_cust.loc[ords_prods_cust['state'].isin(West), 'region'] = 'West'

In [11]:
ords_prods_cust['region'].value_counts(dropna = False)

region
South        10791885
West          8292913
Midwest       7597325
Northeast     5722736
Name: count, dtype: int64

In [12]:
# creating a crosstab between spending_flag and the different regions
crosstab = pd.crosstab(ords_prods_cust['region'], ords_prods_cust['spending_flag'], dropna = False)

In [13]:
crosstab

spending_flag,High spender,Low spender
region,,
Midwest,155975,7441350
Northeast,108225,5614511
South,209691,10582194
West,160354,8132559


## 5. Question 4
#### Create an exclusion flag for low-activity customers and exclude them from the data

In [14]:
# creating exclusion flag for customers with less than 5 orders
ords_prods_cust.loc[ords_prods_cust['max_order'] < 5, 'customer_activity'] = 'low activity customer'

In [15]:
# creating flag for customers with more than 5 orders
ords_prods_cust.loc[ords_prods_cust['max_order'] >= 5, 'customer_activity'] = 'active customer'

In [16]:
ords_prods_cust['customer_activity'].value_counts(dropna = False)

customer_activity
active customer          30964564
low activity customer     1440295
Name: count, dtype: int64

In [17]:
# creating subset of low activity customers
df_low_activity = ords_prods_cust.loc[ords_prods_cust['customer_activity'] == 'low activity customer']

In [18]:
df_low_activity.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_label,busiest_day,busiest_period_of_day,max_order,loyalty_flag,mean_order,spending_flag,median_order,order_frequency_flag,gender,state,age,date_joined,number_of_dependants,marital_status,income,region,customer_activity
1510,520620,120,1,3,11,NaN,196,2,0,Soda,77,7,9.0,Mid-range product,Slowest days,Most orders,3,New customer,9.385714,Low spender,19.0,Regular customer,Female,Kentucky,54,3/2/2017,2,married,99219,South,low activity customer
1511,3273029,120,3,2,8,19.0,196,2,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Average orders,3,New customer,9.385714,Low spender,19.0,Regular customer,Female,Kentucky,54,3/2/2017,2,married,99219,South,low activity customer
1512,520620,120,1,3,11,NaN,46149,1,0,Zero Calorie Cola,77,7,13.4,Mid-range product,Slowest days,Most orders,3,New customer,9.385714,Low spender,19.0,Regular customer,Female,Kentucky,54,3/2/2017,2,married,99219,South,low activity customer
1513,3273029,120,3,2,8,19.0,46149,1,1,Zero Calorie Cola,77,7,13.4,Mid-range product,Regularly busy,Average orders,3,New customer,9.385714,Low spender,19.0,Regular customer,Female,Kentucky,54,3/2/2017,2,married,99219,South,low activity customer
1514,520620,120,1,3,11,NaN,26348,3,0,Mixed Fruit Fruit Snacks,50,19,3.1,Low-range product,Slowest days,Most orders,3,New customer,9.385714,Low spender,19.0,Regular customer,Female,Kentucky,54,3/2/2017,2,married,99219,South,low activity customer


In [19]:
# exporting low activity df
df_low_activity.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'low_activity_customers.pkl'))

In [20]:
# creating active customers subset
df_active_customers = ords_prods_cust.loc[ords_prods_cust['customer_activity'] == 'active customer']

In [21]:
df_low_activity.shape

(1440295, 31)

In [22]:
df_active_customers.shape

(30964564, 31)

In [23]:
#exporting active customers df
df_active_customers.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'high_activity_customers.pkl'))

## 6. Question 5:
#### Create a profiling variable based on age, income, certain goods in the 'department_id' column, and number of dependents

## 6.1 Creating age flag

In [24]:
df_active_customers['age'].describe()

count    3.096456e+07
mean     4.946803e+01
std      1.848528e+01
min      1.800000e+01
25%      3.300000e+01
50%      4.900000e+01
75%      6.500000e+01
max      8.100000e+01
Name: age, dtype: float64

the minimum age in the active customers df is 18, with the max age being 81, and the average age is 49. Creating age groups (18-29, 30-49, and 50-81 years old)

In [25]:
# creating 18-29 flag
df_active_customers.loc[(df_active_customers['age'] >= 18) & (df_active_customers['age'] <=29), 'age_flag'] = '18-29 years old'

C:\Users\Bees\AppData\Local\Temp\ipykernel_9232\3676541435.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_active_customers.loc[(df_active_customers['age'] >= 18) & (df_active_customers['age'] <=29), 'age_flag'] = '18-29 years old'


In [26]:
# creating 30-49 flag
df_active_customers.loc[(df_active_customers['age'] >= 30) & (df_active_customers['age'] <= 49), 'age_flag'] = '30-49 years old'

In [27]:
# creating 50-81 flag
df_active_customers.loc[(df_active_customers['age'] >= 50) & (df_active_customers['age'] <= 81), 'age_flag'] = '50-81 years old'

In [28]:
# checking the flag
df_active_customers['age_flag'].value_counts().sort_index()

age_flag
18-29 years old     5817603
30-49 years old     9730686
50-81 years old    15416275
Name: count, dtype: int64

## 6.2 Creating income flag

In [29]:
# checking income stats
df_active_customers['income'].describe()

count    3.096456e+07
mean     9.967587e+04
std      4.314187e+04
min      2.590300e+04
25%      6.729200e+04
50%      9.676500e+04
75%      1.281020e+05
max      5.939010e+05
Name: income, dtype: float64

creating three groups, a low income (under 65000), middle income(65000-120000), and high income (over 120000) based on the stats

In [30]:
# creating low income flag 
df_active_customers.loc[df_active_customers['income'] <= 65000, 'income_flag'] = 'Low income'

C:\Users\Bees\AppData\Local\Temp\ipykernel_9232\2479129378.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_active_customers.loc[df_active_customers['income'] <= 65000, 'income_flag'] = 'Low income'


In [31]:
# creating middle income flag
df_active_customers.loc[(df_active_customers['income'] > 65000) & (df_active_customers['income'] <= 120000), 'income_flag'] = 'Middle income'

In [32]:
# creating high income flag
df_active_customers.loc[df_active_customers['income'] > 120000, 'income_flag'] = 'High income'

In [33]:
# checking the flag
df_active_customers['income_flag'].value_counts().sort_index()

income_flag
High income       9179513
Low income        7107837
Middle income    14677214
Name: count, dtype: int64

In [34]:
# exporting to continue in second notebook
df_active_customers.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'df_active_customers.pkl'))

### continued on 4.10 IC Coding Etiquette part 2 